In [49]:
import pandas as pd
from statistics import mean, median
import sys
import pprint

In [50]:
pp = pprint.PrettyPrinter(indent=4)

In [51]:
old_result_file = 'median_sbs_delay_all_methods.csv'
ml_delay_file = '../RQ3/final_results/'
ssr_result_file = '../RQ2/ssr_final_results.csv'

In [52]:
batch_sizes = [2, 4, 8, 16]
batching_alg = ['BATCH4', 'BATCHBISECT', 'BATCHSTOP4']

In [53]:
old = pd.read_csv(old_result_file, index_col=0)
gr_results = old[ (old['method']=='gr') & (old['batch_size'] != 32) & ( old['batch_size'] != 1 )]
projects = set(gr_results['project'])

In [54]:
def str_to_list(s):
    if s == '[]':
        return [0]
    l = s[1:-1].split(', ')
    l = [int(x) for x in l]
    return l

In [55]:
gr_delay_lib = {}
ml_delay_lib = {}
ssr_delay_lib = {}

In [56]:
print(len(projects), projects)

20 {'rails.csv', 'cloudify.csv', 'cloud_controller_ng.csv', 'sonarqube.csv', 'orbeon-forms.csv', 'graylog2-server.csv', 'sufia.csv', 'metasploit-framework.csv', 'gradle.csv', 'fog.csv', 'rubinius.csv', 'geoserver.csv', 'puppet.csv', 'loomio.csv', 'opal.csv', 'concerto.csv', 'jruby.csv', 'vagrant.csv', 'heroku.csv', 'open-build-service.csv'}


In [57]:
for p in projects:
    gr_delay_lib[p] = {'BATCHSTOP4':{4:{}, 8:{}, 16:{}}, 'BATCH4':{4:{}}, 'BATCHBISECT':{4:{}, 2:{}, 8:{}, 16:{}}}
    ml_delay_lib[p] = {'BATCHSTOP4':{4:{}, 8:{}, 16:{}}, 'BATCH4':{4:{}}, 'BATCHBISECT':{4:{}, 2:{}, 8:{}, 16:{}}}
    ssr_delay_lib[p] = {'BATCHSTOP4':{4:{}, 8:{}, 16:{}}, 'BATCH4':{4:{}}, 'BATCHBISECT':{1:{}, 2:{}, 4:{}, 8:{}, 16:{}}}

In [58]:
for p in projects:
    
    
    test_file = pd.read_csv('../data/test_data/'+p.split('.')[0]+'_test.csv')
    ml_results = pd.read_csv(ml_delay_file + p.split('.')[0] + '_ml_batching.csv')
    
    length = len(test_file)
    
    #ml batch median delay calculation
    for x in range(len(ml_results)):
        row = ml_results.iloc[x]
        ci = str_to_list(row['ci'])
        b = row['batch_size']
        
        #gr batch median delay calculation
        gr_ci = [0] * length
        i = 0
        gr_delay_list = []
        while i < len(gr_ci):
            b -= 1
            gr_delay_list.append(b)
            if b == 0:
                b = row['batch_size']
            i += 1
        
        gr_delay_lib[p][row['algorithm']][row['batch_size']]['batch_median_delay'] = median(gr_delay_list)
        
        i = 0
        delay_list = []
        while i < len(ci):
            if ci[i] == 0:
                if b >= 1:
                    b -= 1
                    delay_list.append(b)
                elif b == 0:
                    b = row['batch_size']
                    b -= 1
                    delay_list.append(b)
            if ci[i] == 1:
                delay_list.append(0)
                b = row['batch_size']
            i += 1
        ml_delay_lib[p][row['algorithm']][row['batch_size']]['batch_median_delay'] = median(delay_list)

In [59]:
ssr_results = pd.read_csv(ssr_result_file)

In [60]:
for x in range(len(ssr_results)):
    row = ssr_results.iloc[x]
    batch_delays = str_to_list(row['batch_median'])    
    
    ssr_delay_lib[row['project']+'.csv'][row['algorithm']][row['batch_size']]['batch_median_delay'] = median(batch_delays)
    

In [61]:
sbs_median = pd.read_csv(old_result_file)

In [66]:
lines = []
for x in range(len(sbs_median)):
    row = sbs_median.iloc[x]
    if row['method'] == 'sbs':
        lines.append([row['project'], row['method'], row['algorithm'], row['batch_size'], row['builds_reqd'], row['batch_delays'], row['sbs_delays']])
    else:
        if row['method'] == 'ssr':
            d = ssr_delay_lib[row['project']][row['algorithm']][row['batch_size']]['batch_median_delay']
        elif row['method'] == 'ml':
            d = ml_delay_lib[row['project']][row['algorithm']][row['batch_size']]['batch_median_delay']
        elif row['method'] == 'gr':
            d = gr_delay_lib[row['project']][row['algorithm']][row['batch_size']]['batch_median_delay']
        lines.append([row['project'], row['method'], row['algorithm'], row['batch_size'], row['builds_reqd'], d, row['sbs_delays']])
    

In [67]:
df = pd.DataFrame(lines, columns=['project', 'method', 'algorithm', 'batch_size', 'builds_reqd', 'batch_delays', 'sbs_delays'])


In [68]:
df.to_csv('median_sbs&batch_delay_all_methods.csv')